# Neural Networks

##### 可以利用torch.nn包来创建神经网络, nn依靠autograd来定义模型并且对其计算微分. 从nn.Module类派生的子类中会包含模型的layers, 子类的成员函数forward(input)会返回模型的运行结果.

##### 经典的训练神经网络的过程包含以下步骤:

定义具有一些可学习参数(权重)的神经网络

在数据集上创建迭代器

将数据送入到网络中处理

计算loss

对参数进行反向求导

更新参数: weight=weight−lr∗gradient weight = weight - 
lr*gradientweight=weight−lr∗gradient

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 当定义好模型的forward()函数以后, backward()函数就会自动利用autograd机制定义, 无需认为定义.



### 可以通过net.parameters()函数来获取模型中可学习的参数

In [3]:
params = net.parameters()
params = list(params)
print(len(params))
#print(params.size())
print(params[0].size())
print(params[1].size())
print(params[2].size())
print(params[3].size())
print(params[4].size())
print(params[5].size())
print(params[6].size())
print(params[7].size())
print(params[8].size())
print(params[9].size())


10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


### 根据网络结构接受的输入, 想网络中传输数据并获取计算结果

In [14]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.1126,  0.0531, -0.0047, -0.0902, -0.0512, -0.0102, -0.1489, -0.0216,
          0.0790,  0.1087]], grad_fn=<AddmmBackward>)


### 下面的代码可以清空梯度缓存并计算所有需要求导的参数的梯度

In [15]:
net.zero_grad()
out.backward(torch.randn(1,10)) 

### 需要注意的是, 整个torch.nn包只支持mini-batches, 所以对于单个样本, 也需要显示指明batch size=1, 即input第一个维度的值为1

### 也可以对单个样本使用input.unsqueeze(0)来添加一个假的batch dimension.

## Loss Function

一个损失函数往往接收的是一对儿数据 (output, target). 然后根据相应规则计算output和target之间相差多远, 如下所示:

In [17]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1) # 令target和output的shape相同.
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss) # tensor(1.3638, grad_fn=<MseLossBackward>)

tensor(0.5499, grad_fn=<MseLossBackward>)


利用.grad_fn属性, 可以看到关于loss的计算图:

In [18]:
print(loss.grad_fn) # 返回MseLossBackward对象
#input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#      -> view -> linear -> relu -> linear -> relu -> linear
#      -> MSELoss
#      -> loss

因此, 当调用loss.backward()时, 就会计算出所有(requires_grad=True的)参数关于loss的梯度, 并且这些参数都将具有.grad属性来获得计算好的梯度

## BackProp

再利用loss.backward()计算梯度之前, 需要先清空已经存在的梯度缓存(因为PyTorch是基于动态图的, 每迭代一次就会留下计算缓存, 到一下次循环时需要手动清楚缓存), 如果不清除的话, 梯度就换累加(注意不是覆盖).

In [27]:
net.zero_grad()  # 清楚缓存
print(net.conv1.bias.grad) # tensor([0., 0., 0., 0., 0., 0.])

loss.backward()

print(net.conv1.bias.grad) # tensor([ 0.0181, -0.0048, -0.0229, -0.0138, -0.0088, -0.0107])

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0039, -0.0066,  0.0189, -0.0069, -0.0043,  0.0155])


## Update The Weights

最简单的更新方法是按照权重的更新公式:

In [28]:
learning_rate = 0.001
for f in net.parameters():
    f.data.sub_(learning_rate*f.grad.data)

当希望使用一些不同的更新方法如SGD, Adam等时, 可以利用torch.optim包来实现, 如下所示:

In [38]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01) # 创建优化器
optimizer.zero_grad() # 清空缓存
output = net(input)
loss = criterion(output, target)
loss.backward() # 计算梯度
optimizer.step() # 执行一次更新

In [1]:
import torch
from torchvision import models
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [28]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.zeros_(m.bias)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        return x
    
class ConvBlock1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1, 0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.zeros_(m.bias)
        
    def forward(self, x):
        x = self.conv1(x)
        return x
    
class Classifier(nn.Module):
    def __init__(self, num_classes=1000): # <======== modificaition to comply fast.ai
        super().__init__()
        
        self.conv = nn.Sequential(
            ConvBlock(in_channels=3, out_channels=32),
            ConvBlock(in_channels=32, out_channels=64),
            ConvBlock(in_channels=64, out_channels=128),
            ConvBlock(in_channels=128, out_channels=256),
            ConvBlock(in_channels=256, out_channels=512),
            ConvBlock1(in_channels=512, out_channels=1024),
            ConvBlock1(in_channels=1024, out_channels=1024),
            ConvBlock(in_channels=1024, out_channels=512)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # <======== modificaition to comply fast.ai
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(512*2*2, 1024),
            nn.PReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.PReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.conv(x)
        #x = self.avgpool(x)
        #x = self.fc(x)
        return x

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Classifier().to(device)

summary(net, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         ConvBlock-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]          18,496
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
         ConvBlock-8           [-1, 64, 32, 32]               0
            Conv2d-9          [-1, 128, 32, 32]          73,856
      BatchNorm2d-10          [-1, 128, 32, 32]             256
             ReLU-11          [-1, 128, 32, 32]               0
        ConvBlock-12          [-1, 128, 16, 16]               0
           Conv2d-13          [-1, 256, 16, 16]         295,168
      BatchNorm2d-14          [-1, 256,

In [11]:
import random
random.seed(520)

In [28]:
random.randint(1, 128)

84